In [ ]:
import numpy as np
import xarray as xr
from pathlib import Path
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import seaborn as sns

strength_cmap = sns.cubehelix_palette(start=0.5, rot=-0.5, as_cmap=True)

from sdm_eurec4a.visulization import (
    set_custom_rcParams,
    label_from_attrs,
    adjust_lightness_array,
    plot_one_one,
    handler_map_alpha,
)
from sdm_eurec4a.reductions import mean_and_stderror_of_mean

from sdm_eurec4a import slurm_cluster as scluster

default_colors = set_custom_rcParams()
default_dark_colors = adjust_lightness_array(default_colors, 0.75)

In [ ]:
data_dir = Path("/home/m/m301096/CLEO/data/output_v4.0/")

eulerian_data_path = lambda microphysics: data_dir / Path(
    f"{microphysics}/combined/eulerian_dataset_combined.nc"
)
conservation_data_path = lambda microphysics: data_dir / Path(
    f"{microphysics}/combined/conservation_dataset_combined.nc"
)
mean_radius_data_path = lambda microphysics: data_dir / Path(
    f"{microphysics}/combined/mean_radius_combined.nc"
)

In [ ]:
microphysics = (
    "null_microphysics",
    "condensation",
    "collision_condensation",
    "coalbure_condensation_small",
    "coalbure_condensation_large",
)

compare_microphysics = (
    # "null_microphysics",
    "condensation",
    "collision_condensation",
    "coalbure_condensation_small",
    "coalbure_condensation_large",
)

labels: dict = dict(
    zip(
        microphysics,
        [
            "Null",
            "Evap only",
            "Coll",
            "CoalBuRe few",
            "CoalBuRe many",
        ],
    )
)

colors: dict = dict(
    zip(
        microphysics,
        [
            "grey",
            "purple",
            "blue",
            "red",
            "orange",
            "red",
            "orange",
        ],
    )
)
dark_colors: dict = dict(zip(microphysics, adjust_lightness_array(colors.values(), 0.75)))
light_colors: dict = dict(zip(microphysics, adjust_lightness_array(colors.values(), 1.25)))
markers: dict = dict(
    zip(
        microphysics,
        [
            "$*$",
            "1",
            "x",
            "+",
            "2",
            "3",
        ],
    )
)

In [ ]:
client, cluster = scluster.init_dask_slurm_cluster(
    scale=1, processes=8, walltime="00:05:00", memory="32GB"
)

/work/um1487/m301096/conda/envs/sdm_pysd_python312/lib/python3.12/site-packages/distributed/node.py:187: UserWarning: Port 8989 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 36461 instead
  warnings.warn(


#!/usr/bin/env bash

#SBATCH -J dask-worker
#SBATCH -e /scratch/m/m301096/dask_logs/dask-worker-%J.err
#SBATCH -o /scratch/m/m301096/dask_logs/dask-worker-%J.out
#SBATCH -p compute
#SBATCH -A um1487
#SBATCH -n 1
#SBATCH --cpus-per-task=8
#SBATCH --mem=30G
#SBATCH -t 00:05:00

/work/um1487/m301096/conda/envs/sdm_pysd_python312/bin/python -m distributed.cli.dask_worker tcp://10.128.6.250:34687 --name dummy-name --nthreads 1 --memory-limit 3.73GiB --nworkers 8 --nanny --death-timeout 60 --local-directory /scratch/m/m301096/dask_temp --interface ib0



## Data loading

In [ ]:
time_slice = slice(1500, 3490)
chunks = dict(cloud_id=2, gridbox=1)

l = []
for microphysic in ["condensation"]:
    print(microphysic)
    ds_euler_org = xr.open_dataset(eulerian_data_path(microphysic), chunks=chunks)
    ds_euler = ds_euler_org[
        [
            "mass_represented",
            "xi",
            "gridbox_coord3",
            "max_gridbox",
            "liquid_water_content",
            "number_superdroplets",
        ]
    ]

condensation


In [ ]:
cloud_id = np.random.choice(ds_euler["cloud_id"].values, 1)
ds_sel = ds_euler.sel(cloud_id=cloud_id)
gridboxes = np.linspace(0, ds_sel["max_gridbox"].values, 5)
gridboxes = np.append(gridboxes, ds_sel["max_gridbox"].values)
ds_sel = ds_sel.sel(time=time_slice).sel(gridbox=gridboxes, method="nearest").mean("time").compute()

da = ds_sel[["mass_represented", "xi"]]
da = da / da.quantile(0.99, "radius_bins")

ylims = [0, 1]
df = da.drop_vars("cloud_id").to_dataframe()
df = df.reset_index()


# Initialize the FacetGrid object
pal = sns.cubehelix_palette(len(da["gridbox"]), rot=-0.25, light=0.7)
g = sns.FacetGrid(
    df,
    row="gridbox",
    hue="gridbox",
    aspect=7,
    height=0.9,
    palette=pal,
    row_order=np.flip(df["gridbox"].unique()),
)

# Draw the densities in a few steps
g.map(sns.lineplot, "radius_bins", "mass_represented", alpha=1, linewidth=1.5)
g.map(sns.lineplot, "radius_bins", "xi", alpha=1, linewidth=1.5, linestyle="--")

# passing color=None to refline() uses the hue mapping
g.refline(y=0, linewidth=2, linestyle="-", color=None, clip_on=False)


def fill_under_lines(x, color, label, ax=None, alpha=0.2):
    if ax is None:
        ax = plt.gca()
    for line in ax.lines:
        x, y = line.get_xydata().T
        ax.fill_between(x, 0, y, color=line.get_color(), alpha=alpha)


# Define and use a simple function to label the plot in axes coordinates
def label(x, color, label):
    ax = plt.gca()
    ax.text(
        -0.1, 0.2, label, fontweight="bold", color=color, ha="left", va="center", transform=ax.transAxes
    )


# def xscale(x, color, label, lims) :
#     ax = plt.gca()
#     ax.set_xscale("log")
#     ax.set_xlim(1, None)
#     ax.set_ylim(0, 1)

g.map(label, "radius_bins")
# g.map(fill_under_lines, "radius_bins")

# Set the subplots to overlap
g.figure.subplots_adjust(hspace=-0.15)
g.figure.suptitle(f"{cloud_id}")
# Remove axes details that don't play well with overlap
g.set_titles("")
g.set(ylim=ylims, xlim=(10, None), xscale="log")
g.set(yticks=[], ylabel="")
g.despine(bottom=True, left=True)